Efficient solution of Poisson's equation using discrete variable representation basis sets for Car-Parrinello *ab initio* molecular dynamics simulations with cluster boundary conditions

Normalized sinc-DVR function ${u_{\alpha}(x)}$
$$
\begin{equation}
u_{\alpha}(x) = \frac{\sqrt{d}}{\pi}
\frac{\sin\left[\pi(x-x_{\alpha})\right]}{x-x_{\alpha}}
\end{equation}
$$
Grid spacing: $d = \pi\hbar/p_{0}$

Hartree potential:
$$
\begin{equation}
V_{\mathrm{H}}(\mathbf{r}) = \int \mathrm{d}\mathbf{r}
\frac{\rho(\mathbf{r}')}{\left|\mathbf{r}'-\mathbf{r}\right|}
\end{equation}
$$

The term $1/r$ is rewritten using the following integral expression for $r^{-\alpha}$:
$$
\begin{equation}
r^{-\alpha} = \frac{2}{\Gamma(\alpha/2)}
\int_{-\infty}^{\infty} e^{-r^2\,e^{2y}+ay}\,\mathrm{d}y
\end{equation}
$$
Using change of variable to $t = e^{y}$, the identity become:
$$
\begin{equation}
r^{-\alpha} = \frac{2}{\Gamma(\alpha/2)}
\int_{0}^{\infty} e^{-r^2 t^{2}}t^{\alpha-1}\,\mathrm{d}y
\end{equation}
$$
For $\alpha = 1$:
$$
\begin{equation}
\frac{1}{r} = \frac{2}{\sqrt{\pi}}
\int_{0}^{\infty} e^{-r^2 t^{2}}\,\mathrm{d}y
\end{equation}
$$

Using the identity above, the equation for Hartree potential can be written as:
$$
\begin{equation}
V_{\mathrm{H}}(\mathbf{r}) =
\frac{2}{\sqrt{\pi}}\int_{0}^{\infty}\int\,\mathrm{d}\mathbf{r}'
e^{-t^2\left(\left| \mathbf{r}-\mathbf{r}' \right|^2\right)}
\rho(\mathbf{r}')
\end{equation}
$$

Using direct product of one-dimensional sinc-DVR functions as
$$
\begin{equation}
\rho(\mathbf{r}) = \sum_{\alpha\beta\gamma}
d_{\alpha\beta\gamma} u_{\alpha}(x) u_{\beta}(y) u_{\gamma}(z)
\end{equation}
$$
where $d_{\alpha\beta\gamma}$ is a set of expansion coefficients for $\mathrm{\rho}(\mathbf{r})$

# Testing identity

Test the identity:
$$
\begin{equation}
\frac{1}{r} = \frac{2}{\sqrt{\pi}}
\int_{0}^{\infty} e^{-r^2 t^{2}}\,\mathrm{d}y
\end{equation}
$$

In [1]:
using FastGaussQuadrature
x, w = gausslegendre(10)

([-0.973907, -0.865063, -0.67941, -0.433395, -0.148874, 0.148874, 0.433395, 0.67941, 0.865063, 0.973907], [0.0666713, 0.149451, 0.219086, 0.269267, 0.295524, 0.295524, 0.269267, 0.219086, 0.149451, 0.0666713])

In [9]:
# Given the lower and upper limits of integration x1 and x2, and given n, this routine returns
# arrays x[1..n] and w[1..n] of length n, containing the abscissas and weights of the Gauss-
# Legendre n-point quadrature formula.
function gauleg( x1::Float64, x2::Float64, n::Int64)
    EPS = 3.0E-11 # Relative precision
    # int m, j, i;
    # double z1, z, xm, xl, pp, p3, p2, p1; // High precision is a good idea for this routine.
    # Output
    x = Array{Float64}(n)
    w = Array{Float64}(n)
    m = round( Int64, (n + 1)/2 )      # The roots are symmetric in the interval, so
    xm = 0.5*(x2 + x1) # we only have to find half of them.
    xl = 0.5*(x2 - x1)
    z1 = 100.0  # some initial number to get the while loop enter the first time
    pp = 0.0    # make pp visible outside for loop
    for i = 1:m # Loop over the desired roots.
        z = cos( pi*(i-0.25)/(n+0.5) )
        # Starting with the above approximation to the ith root, we enter the main loop of
        # refinement by Newton’s method.
        while abs(z-z1) > EPS
            p1 = 1.0;
            p2 = 0.0;
            for j=1:n # Loop up the recurrence relation to get the
                p3 = p2       # Legendre polynomial evaluated at z.
                p2 = p1
                p1 = ( (2.0*j-1.0)*z*p2-(j-1.0)*p3)/j
            end
            # p1 is now the desired Legendre polynomial. We next compute pp, its derivative,
            # by a standard relation involving also p2, the polynomial of one lower order.
            pp = n*(z*p1-p2)/(z*z-1.0)
            z1 = z
            z  = z1 - p1/pp # Newton’s method.
        end # while
        #
        x[i] = xm - xl*z;                  # Scale the root to the desired interval,
        x[n-i+1] = xm + xl*z;              # and put in its symmetric counterpart.
        w[i] = 2.0*xl/((1.0-z*z)*pp*pp) # Compute the weight
        w[n-i+1] = w[i]                 # and its symmetric counterpart.
    end
  return x, w
end
function t_sampling( num_points1::Int64, num_points2::Int64,
                     t_i::Float64, t_l::Float64, t_f::Float64 )
    # New method by Sundholm (JCP 132, 024102, 2010).
    # Within two-divided regions ([t_i,t_l], [t_l,t_f]),
    # num_points1, num_points2 quadrature points are made, respectively.
    t_values = Array{Float64}(num_points1 + num_points2)
    w_t = Array{Float64}(num_points1 + num_points2)
    # Linear coord region:  [t_i, t_l]
    x_leg, w_leg = gauleg(t_i, t_l, num_points1);
    for j=1:num_points1
        t_values[j] = x_leg[j]
        w_t[j]      = w_leg[j]*2.0/sqrt(pi)
    end
    # Logarithmic coord region: [t_l, t_f]
    x_leg, w_leg = gauleg( log(t_l), log(t_f), num_points2)
    # Return the log-coord-partitioned points back to linear t-space.
    s_p = 0.0
    w_p = 0.0
    for j=1:num_points2
        s_p = x_leg[j]
        w_p = w_leg[j]
        x_leg[j] = exp(s_p)
        w_leg[j] = w_p * exp(s_p)
    end
    for j=1:num_points2
        t_values[num_points1+j] = x_leg[j]
        w_t[num_points1+j] = w_leg[j]*2.0/sqrt(pi)
    end
    return t_values, w_t
end


t_sampling (generic function with 1 method)

In [3]:
function integrand(t, r)
    return exp(-t^2 * r^2)
end

integrand (generic function with 1 method)

Using the approach given in `HeterogeneousHartree` by SungHwan Choi

In [38]:
num_points1 = 50
num_points2 = 100
NptsInteg = num_points1 + num_points2
t_i = 0.0
t_l = 1.0
t_f = 10000.0
t, w = t_sampling( num_points1, num_points2, t_i, t_l, t_f)
ss = 0.0
r = 10.0
for i = 1:NptsInteg
    ss = ss + w[i]*integrand(t[i], r)
end
@printf("integral = %18.10f", ss)

integral =       0.1000000000

Using the usual Gaussian integration (without using logaritmic grid)

In [36]:
NptsInteg = 500
t_i = 0.0
t_f = 10000.0
t, w = gauleg( t_i, t_f, NptsInteg )
ss = 0.0
r = 10.0
for i = 1:NptsInteg
    ss = ss + w[i]*integrand(t[i], r)
end
@printf("integral = %18.10f", ss*2.0/sqrt(pi))

integral =       0.1198197804